In [318]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [319]:
with open('./all_docs_results.json') as json_file:
    predictions = json.load(json_file)
    
with open('./project-12-at-2021-07-15-14-39-0020e5b8.json') as json_file:
    ground_truth = json.load(json_file)

## Predictions
### Exploración

In [320]:
predictions['11821']['CSF_match_results'].keys()

dict_keys(['state_csf', 'society_type_csf', 'company_name_csf'])

In [321]:
predictions['11821']['ROIS_qa_results'].keys()

dict_keys(['notario', 'date'])

In [322]:
society_type_csf = predictions['11821']['CSF_match_results']['society_type_csf'][3]
company_name_csf = predictions['11821']['CSF_match_results']['company_name_csf'][3]
notario = predictions['11821']['ROIS_qa_results']['notario'][3]
date = predictions['11821']['ROIS_qa_results']['date'][3]

## Armo un df de predicciones

In [323]:
campos = []
for i in range(len(predictions)):
    for k in predictions['11821']['CSF_match_results'].keys():
        campos.append(k)
    for k in predictions['11821']['ROIS_qa_results'].keys():
        campos.append(k)
        
campos_unique = sorted(set(campos))
campos_unique

['company_name_csf', 'date', 'notario', 'society_type_csf', 'state_csf']

In [324]:
df_preds = pd.DataFrame(columns = ['documento'])
for c in campos_unique:
    df_preds[c] = []
    
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf


In [325]:
i = 0
for k in predictions.keys():
    
    df_preds.loc[i, 'documento'] = k
    
    # a veces puede ser que 'CSF_match_results' o 'ROIS_qa_results' estén vacíos
    try:
        for j in predictions[k]['CSF_match_results'].keys():
            try:
                df_preds.loc[i, j] = predictions[k]['CSF_match_results'][j][3]
            except:
                continue
    except:
        continue
        
    try:    
        for j in predictions[k]['ROIS_qa_results'].keys():
            try:
                df_preds.loc[i, j] = predictions[k]['ROIS_qa_results'][j][3]
            except:
                continue
    except:
        continue
        
    i = i+1
            
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,GRUPO CYCMP,veintiumo de julio del dos mil ocho,MAURICIO MIER PADRÓN,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,San Luis Potosi
1,12078,BITUBI CONSULTING,trece de febrero,JORGE JULIÁN PEREYRA ESCUDERO,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,Quintana Roo
2,12192,DESARROLLO DE SOFTWARE MONOBITS,22 de noviembre de 2012,YO,Sociedad de Responsabilidad Limitada de Capita...,BAJA CALIFORNIA
3,12264,CHARVEL Y PEZUELA GROUP,dos de octubre,NaN,SOCIEDAD ANONIMA DE CAPITAL VARIABLE,CIUDAD DE MÉXICO
4,12286,EXELENCIA EJECUTIVA PREMIUM,NaN,RAFAEL OVARRUBIAS FLORES,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,JALISCO


De todos los campos que puedo encontrar en las actas, no todos los puedo validar, sólo éstos de arriba

## Ground truth
### Exploración

In [326]:
# este tiene la boleta de inscripción
ground_truth[0]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.866666666666667,
   'y': 11.902834008097166,
   'width': 83.46666666666667,
   'height': 36.51821862348178,
   'rotation': 0,
   'rectanglelabels': ['boleta_inscripcion']},
  'id': 'nVvt8xx7hA',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'choices': ['yes']},
  'id': 'r_7Rb71M-M',
  'from_name': 'has_boleta_inscripcion',
  'to_name': 'image',
  'type': 'choices'}]

In [327]:
# este tiene el cuadro accionario y el apoderado
ground_truth[1]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.432706222865413,
   'y': 39.19156414762742,
   'width': 57.45296671490593,
   'height': 7.0298769771529,
   'rotation': 0,
   'rectanglelabels': ['sh_table_lines']},
  'id': 'ANsPu48gOp',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'text': ['Jorge Cañedo Osuna']},
  'id': 'asa879Jesq',
  'from_name': 'apoderado',
  'to_name': 'image',
  'type': 'textarea'}]

In [328]:
# este tiene varios campos
# lo que me interesa es todo lo que está en "result"
result_lista = ground_truth[2]['annotations'][0]['result']

result_lista

[{'value': {'text': ['Miguel Enrique Martinez Serrato']},
  'id': 'KdMY7vBSEi',
  'from_name': 'notario',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Compass4PL']},
  'id': 'hQFu6ebrse',
  'from_name': 'razon_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['S de RL de cv']},
  'id': 'P7GI5OarC-',
  'from_name': 'denominacion_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['27/06/2015']},
  'id': '_LjHuXTpo8',
  'from_name': 'fecha_inscripcion',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['12236']},
  'id': 'cSq9FwAKry',
  'from_name': 'instrumento_publico',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Navojoa Sonora Mexico']},
  'id': 'XAR0_Uu-DH',
  'from_name': 'direccion',
  'to_name': 'image',
  'type': 'textarea'}]

In [329]:
id = ground_truth[1]['id']
id

7887

In [330]:
documento = ground_truth[1]['file_upload'].split('-')[0]
documento

'16519'

In [331]:
# la página a veces tiene un "_" seguido de otras cosas
pagina = ground_truth[1]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
pagina

'014'

In [332]:
# acá no hay campos que me interesen
ground_truth[0]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.866666666666667,
   'y': 11.902834008097166,
   'width': 83.46666666666667,
   'height': 36.51821862348178,
   'rotation': 0,
   'rectanglelabels': ['boleta_inscripcion']},
  'id': 'nVvt8xx7hA',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'choices': ['yes']},
  'id': 'r_7Rb71M-M',
  'from_name': 'has_boleta_inscripcion',
  'to_name': 'image',
  'type': 'choices'}]

In [333]:
# acá me interesa el campo "apoderado"
ground_truth[1]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.432706222865413,
   'y': 39.19156414762742,
   'width': 57.45296671490593,
   'height': 7.0298769771529,
   'rotation': 0,
   'rectanglelabels': ['sh_table_lines']},
  'id': 'ANsPu48gOp',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'text': ['Jorge Cañedo Osuna']},
  'id': 'asa879Jesq',
  'from_name': 'apoderado',
  'to_name': 'image',
  'type': 'textarea'}]

Vemos que el primer key es "value".

Acá hay varios campos que podría levantar, aunque no son los 4 que tengo en el otro archivo. También empiezan con "value":

In [334]:
ground_truth[2]['annotations'][0]['result']

[{'value': {'text': ['Miguel Enrique Martinez Serrato']},
  'id': 'KdMY7vBSEi',
  'from_name': 'notario',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Compass4PL']},
  'id': 'hQFu6ebrse',
  'from_name': 'razon_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['S de RL de cv']},
  'id': 'P7GI5OarC-',
  'from_name': 'denominacion_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['27/06/2015']},
  'id': '_LjHuXTpo8',
  'from_name': 'fecha_inscripcion',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['12236']},
  'id': 'cSq9FwAKry',
  'from_name': 'instrumento_publico',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Navojoa Sonora Mexico']},
  'id': 'XAR0_Uu-DH',
  'from_name': 'direccion',
  'to_name': 'image',
  'type': 'textarea'}]

## Armo un df de ground truth

In [335]:
# quiero saber cuáles son todos los campos posibles para hacer columnas en el df
campos = []
for i in range(len(ground_truth)):
    lista = ground_truth[i]['annotations'][0]['result']
    
    for l in lista:
        # acá salteo todos los que no tengan el formato que me interesa
        try:
            campos.append(l['from_name'])
        except:
            continue
            
campos_unique = sorted(set(campos))

In [336]:
for c in campos_unique:
    print(c)

apoderado
denominacion_social
direccion
fecha_inscripcion
has_boleta_inscripcion
instrumento_publico
label
notario
razon_social


In [337]:
# hago una nueva versión del df con una columna por campo
df_truth = pd.DataFrame(columns = ['id', 'documento', 'pagina'])
for c in campos_unique:
    df_truth[c] = []
    
# df.head()

In [338]:
# ahora trato de llenar todos esos campos, cuando es posible

for i in range(len(ground_truth)):
#     print(i)
    identity = ground_truth[i]['id']
    documento = ground_truth[i]['file_upload'].split('-')[0]
    pagina = ground_truth[i]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
    
    lista = ground_truth[i]['annotations'][0]['result']

    for l in lista:
        df_truth.loc[i, 'id'] = identity
        df_truth.loc[i, 'documento'] = documento
        df_truth.loc[i, 'pagina'] = pagina
        
        try:
            df_truth.loc[i, l['from_name']] = l['value']['text'][0]
        except:
            continue
    
# df_truth.head()

In [339]:
# ordeno por documento y página
df_truth = df_truth.sort_values(['documento', 'pagina'])
df_truth.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
1276,6631,10227,009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1272,6635,10227,019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,6908,11821,000,NaN,sa de cv,san luis potosi,21/07/2008,NaN,39413,NaN,mauricio mier padron,grupo cycme
95,6897,11821,011,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,6910,11821,015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [340]:
# convierto pagina a int
df_truth['pagina'] = df_truth['pagina'].astype(int)
df_truth.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
1276,6631,10227,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1272,6635,10227,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,6908,11821,0,NaN,sa de cv,san luis potosi,21/07/2008,NaN,39413,NaN,mauricio mier padron,grupo cycme
95,6897,11821,11,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,6910,11821,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Comparación entre los 2 dataframes
Hago un df nuevo donde dejo sólo los campos que quiero comparar, y colapso todas las páginas.

Quizás es más fácil construirlo directamente de esa forma. En este caso los campos "id" y "pagina" ya no tienen sentido.

In [341]:
# este df lo lleno juntando todas las páginas
# para eso necesito iterar por documento. Hago una lista de todos los docs que hay

docs = []
for i in range(len(ground_truth)):
    documento = ground_truth[i]['file_upload'].split('-')[0]
    docs.append(documento)
    
docs_unique = sorted(list(set(docs)))

In [342]:
df_truth_compact = pd.DataFrame(docs_unique, columns = ['documento'])
# df_truth_compact.head()

In [343]:
# i=0
for d in docs_unique:
    for g in ground_truth:
        documento = g['file_upload'].split('-')[0]
        if documento == d:
            lista = g['annotations'][0]['result']

            for l in lista:
                try:
                    df_truth_compact.loc[df_truth_compact['documento'] == d, l['from_name']] = l['value']['text'][0]
#                     print(i, documento, l['from_name'], l['value']['text'][0])
                except:
                    continue
#             i=i+1
            
df_truth_compact.head()

,documento,apoderado,notario,razon_social,denominacion_social,direccion,fecha_inscripcion,instrumento_publico
0,10227,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11821,raymundo garza elizondo,mauricio mier padron,grupo cycme,sa de cv,san luis potosi,21/07/2008,39413
2,12078,mauricio zúñiga de la cerda,jorge julian pereyra escudero,bitubi consulting,sa de cv,cancun quintana roo,27/03/2018,360
3,12192,cesar higuera chavez,virgilio cesar valdez portales,desarrollo de software monobits,s de rl de cv,"mexicali, baja california",21/01/2016,107826
4,12264,lilia guadalupe gomez pezuela gamboa,Jorge Rábago Ordóñez,charvel y pezuela group,sa de cv,"ciudad de mexico, distrito federal",02/10/2012,437


Me quiero quedar sólo con las columnas que voy a poder validar:

In [344]:
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,GRUPO CYCMP,veintiumo de julio del dos mil ocho,MAURICIO MIER PADRÓN,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,San Luis Potosi
1,12078,BITUBI CONSULTING,trece de febrero,JORGE JULIÁN PEREYRA ESCUDERO,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,Quintana Roo
2,12192,DESARROLLO DE SOFTWARE MONOBITS,22 de noviembre de 2012,YO,Sociedad de Responsabilidad Limitada de Capita...,BAJA CALIFORNIA
3,12264,CHARVEL Y PEZUELA GROUP,dos de octubre,NaN,SOCIEDAD ANONIMA DE CAPITAL VARIABLE,CIUDAD DE MÉXICO
4,12286,EXELENCIA EJECUTIVA PREMIUM,NaN,RAFAEL OVARRUBIAS FLORES,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,JALISCO


In [345]:
df_truth_compact = df_truth_compact[['documento', 'razon_social', 'fecha_inscripcion', 'notario', 'denominacion_social', 'direccion']]
df_truth_compact.head()

,documento,razon_social,fecha_inscripcion,notario,denominacion_social,direccion
0,10227,NaN,NaN,NaN,NaN,NaN
1,11821,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
2,12078,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
3,12192,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
4,12264,charvel y pezuela group,02/10/2012,Jorge Rábago Ordóñez,sa de cv,"ciudad de mexico, distrito federal"


### Normalizo el texto de las predicciones
Lower case, sin acentos.

In [346]:
# convierto todo a lower case
for col in list(df_preds.drop(['documento'], axis=1)):
    df_preds[col] = df_preds[col].str.lower()
    
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padrón,sociedad anónima de capital variable,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julián pereyra escudero,sociedad anónima de capital variable,quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,sociedad de responsabilidad limitada de capita...,baja california
3,12264,charvel y pezuela group,dos de octubre,NaN,sociedad anonima de capital variable,ciudad de méxico
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sociedad anónima de capital variable,jalisco


In [347]:
# elimino acentos
for col in list(df_preds.drop(['documento'], axis=1)):
    df_preds[col] = df_preds[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sociedad anonima de capital variable,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sociedad anonima de capital variable,quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,sociedad de responsabilidad limitada de capita...,baja california
3,12264,charvel y pezuela group,dos de octubre,NaN,sociedad anonima de capital variable,ciudad de mexico
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sociedad anonima de capital variable,jalisco


Se me ocurre que society type se puede estandarizar a mano (debe haber pocos valores)

df_truth_compact también tiene mayúsculas:

In [348]:
# convierto todo a lower case
for col in list(df_truth_compact.drop(['documento'], axis=1)):
    df_truth_compact[col] = df_truth_compact[col].str.lower()

In [349]:
df_preds['society_type_csf'].value_counts()

sociedad anonima de capital variable                           11
sociedad de responsabilidad limitada de capital variable        2
sociedad anonima promotora de inversion de capital variable     1
Name: society_type_csf, dtype: int64

In [350]:
# veo cuáles son las otras abreviaciones
df_truth_compact['denominacion_social'].value_counts()

sa de cv                22
s de rl de cv            3
scp                      1
sas de cv                1
sapi de cv sofom enr     1
Name: denominacion_social, dtype: int64

In [351]:
df_preds = df_preds.replace(['sociedad anonima de capital variable'],'sa de cv')
df_preds = df_preds.replace(['sociedad de responsabilidad limitada de capital variable'],'s de rl de cv')
df_preds = df_preds.replace(['sociedad anonima promotora de inversion de capital variable'],'sapi de cv')

df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco


In [352]:
df_truth_compact.head()

,documento,razon_social,fecha_inscripcion,notario,denominacion_social,direccion
0,10227,NaN,NaN,NaN,NaN,NaN
1,11821,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
2,12078,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
3,12192,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
4,12264,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal"


## Estandarizar fechas
Veo qué pinta tienen

In [353]:
df_preds['date'].value_counts()

22 de noviembre de 2012                  2
once de julio del ano                    1
dos de octubre                           1
septiembre                               1
mayos                                    1
trece de febrero                         1
veintiumo de julio del dos mil ocho      1
doce de septiembre                       1
13 trece de octubre de 2018 dieciocho    1
Name: date, dtype: int64

No sé ni por dónde empezar. Capaz puedo empezar matcheando otras cosas. Denominacion_social es fácil.

Creo que me conviene hacer un join de los dos df, para recorrer todos los documentos. Sería un inner join, porque me interesan sólo los documentos que están presentes en ambos datasets.

### Estandarizo los nombres de las columnas:

In [354]:
#df_preds.rename(columns = {'date':'date_csf', 'notario':'notario_csf'}, inplace = True)
#df_truth_compact.rename(columns = {'razon_social':'company_name_pred', 'fecha_inscripcion':'date_pred', 'notario':'notario_pred', 'denominacion_social':'society_type_pred', 'direccion':'state_pred'}, inplace = True)

df_preds.rename(columns = {'date':'date_pred', 'notario':'notario_pred'}, inplace = True)
df_truth_compact.rename(columns = {'razon_social':'company_name', 'fecha_inscripcion':'date', 'notario':'notario', 'denominacion_social':'society_type', 'direccion':'state'}, inplace = True)

In [355]:
df_preds.head()

,documento,company_name_csf,date_pred,notario_pred,society_type_csf,state_csf
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco


In [356]:
df_truth_compact.head()

,documento,company_name,date,notario,society_type,state
0,10227,NaN,NaN,NaN,NaN,NaN
1,11821,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
2,12078,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
3,12192,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
4,12264,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal"


In [357]:
df_merge = pd.merge(df_preds, df_truth_compact, on='documento', how='inner')
df_merge.head()

,documento,company_name_csf,date_pred,notario_pred,society_type_csf,state_csf,company_name,date,notario,society_type,state
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal"
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco,exelencia ejecutiva premium,13/07/2011,rafael covarrubias flores,sa de cv,"guadalajara, jalisco"


## Ahora falta estandarizar las fechas

## Matcheo usando la métrica de RapidFuzz

In [358]:
from rapidfuzz import fuzz

In [359]:
N = df_merge.shape[0]

for i in range(N):
    try:
        df_merge.loc[i, 'society_type_match'] = fuzz.partial_ratio(df_merge.loc[i, 'society_type_csf'], df_merge.loc[i, 'society_type'])
    except:
        df_merge.loc[i, 'society_type_match'] = np.nan
        
    try:
        df_merge.loc[i, 'company_name_match'] = fuzz.partial_ratio(df_merge.loc[i, 'company_name_csf'], df_merge.loc[i, 'company_name'])
    except:
        df_merge.loc[i, 'company_name_match'] = np.nan
        
    try:
        df_merge.loc[i, 'notario_match'] = fuzz.partial_ratio(df_merge.loc[i, 'notario_pred'], df_merge.loc[i, 'notario'])
    except:
        df_merge.loc[i, 'notario_match'] = np.nan
        
    try:
        df_merge.loc[i, 'state_match'] = fuzz.partial_ratio(df_merge.loc[i, 'state_csf'], df_merge.loc[i, 'state'])
    except:
        df_merge.loc[i, 'state_match'] = np.nan
    
df_merge.head()

,documento,company_name_csf,date_pred,notario_pred,society_type_csf,state_csf,company_name,date,notario,society_type,state,society_type_match,company_name_match,notario_match,state_match
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi,100.0,90.909091,100.000000,100.0
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo,100.0,100.000000,100.000000,100.0
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california",100.0,100.000000,50.000000,100.0
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal",100.0,100.000000,NaN,100.0
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco,exelencia ejecutiva premium,13/07/2011,rafael covarrubias flores,sa de cv,"guadalajara, jalisco",100.0,100.000000,95.833333,100.0


Me llama mucho la atención que en **state matchea siempre 100**, a pesar de que muchas veces falta la ciudad. Supongo que el partial ratio funciona así.

También me sorprende que el doc 12192 tiene un **score de 50 en notario**, cuando la diferencia es enorme.

## En qué páginas aparece cada campo

In [360]:
df_truth.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
1276,6631,10227,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1272,6635,10227,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,6908,11821,0,NaN,sa de cv,san luis potosi,21/07/2008,NaN,39413,NaN,mauricio mier padron,grupo cycme
95,6897,11821,11,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,6910,11821,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [361]:
df_truth[df_truth["apoderado"].notnull()].head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
95,6897,11821,11,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,6930,12078,8,mauricio zúñiga de la cerda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,6943,12192,8,cesar higuera chavez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,6995,12264,11,lilia guadalupe gomez pezuela gamboa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,7003,12286,12,arturo uriel tavarez lópez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [362]:
# list(df_truth.loc[df_truth["apoderado"].notnull(), 'pagina'])

In [363]:
# guardo las páginas de cada campo en un diccionario
paginas = {}

for column in df_truth.drop(['id', 'documento', 'pagina'], axis=1):
    paginas[column] = list(df_truth.loc[df_truth[column].notnull(), 'pagina'])
    
# paginas

In [364]:
# grafico las distribuciones
import plotly.express as px

for k in paginas.keys():
    try:
        fig = px.histogram(paginas[k], x=paginas[k], title=k, width=600, height=400)
        fig.show()
    except:
        continue

## Distribución de scores para cada campo

In [365]:
df_merge.head()

,documento,company_name_csf,date_pred,notario_pred,society_type_csf,state_csf,company_name,date,notario,society_type,state,society_type_match,company_name_match,notario_match,state_match
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi,100.0,90.909091,100.000000,100.0
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo,100.0,100.000000,100.000000,100.0
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california",100.0,100.000000,50.000000,100.0
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal",100.0,100.000000,NaN,100.0
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco,exelencia ejecutiva premium,13/07/2011,rafael covarrubias flores,sa de cv,"guadalajara, jalisco",100.0,100.000000,95.833333,100.0


In [366]:
match_cols = [col for col in df_merge.columns if 'match' in col]

for column in match_cols:
    fig = px.histogram(df_merge, x=column, title=column, width=600, height=400)
    fig.show()

## Top 5 matches por campo
### A los NaN los ignora. Queremos considerarlos como un mal match?
Si es así debería asignarles un 0.

In [367]:
# df_merge.head()

In [368]:
# for column in match_cols:
#     print((df_merge[['documento', column]].sort_values([column], ascending=[False])).iloc[:5])

In [369]:
# quiero mostrar los pares de columnas matcheadas
# para eso defino los pares en un dict:

pares = {'society_type_match':['society_type_csf', 'society_type'], 'company_name_match':['company_name_csf', 'company_name'], 'notario_match':['notario_pred', 'notario'], 'state_match':['state_csf', 'state']}

In [370]:
for p in pares.keys():
    print(p, '- Mejores 5 matches')
    doc_idx = ((df_merge[['documento', p]].sort_values([p], ascending=[False])).iloc[:5])['documento']
        
    print(df_merge[df_merge['documento'].isin(doc_idx)][['documento', pares[p][0], pares[p][1], p]].sort_values(by=[p], ascending=False))
    print()

society_type_match - Mejores 5 matches
  documento society_type_csf   society_type  society_type_match
0     11821         sa de cv       sa de cv               100.0
1     12078         sa de cv       sa de cv               100.0
2     12192    s de rl de cv  s de rl de cv               100.0
3     12264         sa de cv       sa de cv               100.0
4     12286         sa de cv       sa de cv               100.0

company_name_match - Mejores 5 matches
  documento                 company_name_csf                     company_name  \
1     12078                bitubi consulting                bitubi consulting   
2     12192  desarrollo de software monobits  desarrollo de software monobits   
3     12264          charvel y pezuela group          charvel y pezuela group   
4     12286      exelencia ejecutiva premium      exelencia ejecutiva premium   
6     12523      grupo empresarial geldreich      grupo empresarial geldreich   

   company_name_match  
1               100.0  
2 

## Peores 5 matches por campo

In [371]:
for p in pares.keys():
    print(p, '- Peores 5 matches')
    doc_idx = ((df_merge[['documento', p]].sort_values([p], ascending=[True])).iloc[:5])['documento']
        
    print(df_merge[df_merge['documento'].isin(doc_idx)][['documento', pares[p][0], pares[p][1], p]].sort_values(by=[p]))
    print()

society_type_match - Peores 5 matches
  documento society_type_csf   society_type  society_type_match
0     11821         sa de cv       sa de cv               100.0
1     12078         sa de cv       sa de cv               100.0
2     12192    s de rl de cv  s de rl de cv               100.0
3     12264         sa de cv       sa de cv               100.0
4     12286         sa de cv       sa de cv               100.0

company_name_match - Peores 5 matches
   documento                        company_name_csf  \
0      11821                             grupo cycmp   
8      12809               comercializadora algadisa   
5      12522                         indimex trading   
11     14624  construcciones electromecanicas pliego   
1      12078                       bitubi consulting   

                               company_name  company_name_match  
0                               grupo cycme           90.909091  
8                 comercializadora algaliza           92.000000  
5   

## En cuántos casos se pudo detectar (ratio>90)

In [372]:
df_merge[match_cols].head()

,society_type_match,company_name_match,notario_match,state_match
0,100.0,90.909091,100.000000,100.0
1,100.0,100.000000,100.000000,100.0
2,100.0,100.000000,50.000000,100.0
3,100.0,100.000000,NaN,100.0
4,100.0,100.000000,95.833333,100.0


In [373]:
df_merge[match_cols].ge(90)

,society_type_match,company_name_match,notario_match,state_match
0,True,True,True,True
1,True,True,True,True
2,True,True,False,True
3,True,True,False,True
4,True,True,True,True
5,True,True,False,False
6,True,True,False,True
7,True,True,False,True
8,True,True,False,False
9,True,True,False,True


In [374]:
for c in match_cols:
    print(c)
    print(df_merge[c].ge(90).value_counts().loc[True])

society_type_match
14
company_name_match
14
notario_match
5
state_match
11


### Sobre un total de cuántos datos?

In [375]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   documento           14 non-null     object 
 1   company_name_csf    14 non-null     object 
 2   date_pred           10 non-null     object 
 3   notario_pred        10 non-null     object 
 4   society_type_csf    14 non-null     object 
 5   state_csf           14 non-null     object 
 6   company_name        14 non-null     object 
 7   date                14 non-null     object 
 8   notario             14 non-null     object 
 9   society_type        14 non-null     object 
 10  state               13 non-null     object 
 11  society_type_match  14 non-null     float64
 12  company_name_match  14 non-null     float64
 13  notario_match       10 non-null     float64
 14  state_match         13 non-null     float64
dtypes: float64(4), object(11)
memory usage: 2.3+ KB


In [376]:
total = 14
for c in match_cols:
    print(c)
    matches = df_merge[c].ge(90).value_counts().loc[True]
    print(matches/total)

society_type_match
1.0
company_name_match
1.0
notario_match
0.35714285714285715
state_match
0.7857142857142857


In [377]:
df_preds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   documento         14 non-null     object
 1   company_name_csf  14 non-null     object
 2   date_pred         10 non-null     object
 3   notario_pred      10 non-null     object
 4   society_type_csf  14 non-null     object
 5   state_csf         14 non-null     object
dtypes: object(6)
memory usage: 1.3+ KB


In [378]:
df_truth.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 1276 to 115
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      144 non-null    object 
 1   documento               144 non-null    object 
 2   pagina                  144 non-null    int64  
 3   apoderado               27 non-null     object 
 4   denominacion_social     28 non-null     object 
 5   direccion               25 non-null     object 
 6   fecha_inscripcion       25 non-null     object 
 7   has_boleta_inscripcion  0 non-null      float64
 8   instrumento_publico     25 non-null     object 
 9   label                   0 non-null      float64
 10  notario                 26 non-null     object 
 11  razon_social            28 non-null     object 
dtypes: float64(2), int64(1), object(9)
memory usage: 14.6+ KB


In [379]:
df_notario = df_merge[['notario', 'notario_pred', 'notario_match']].copy()
df_notario.head()

,notario,notario_pred,notario_match
0,mauricio mier padron,mauricio mier padron,100.000000
1,jorge julian pereyra escudero,jorge julian pereyra escudero,100.000000
2,virgilio cesar valdez portales,yo,50.000000
3,jorge rábago ordóñez,NaN,NaN
4,rafael covarrubias flores,rafael ovarrubias flores,95.833333


In [380]:
df_notario['success'] = df_notario['notario_match'].ge(90)

In [381]:
df_notario.head()

,notario,notario_pred,notario_match,success
0,mauricio mier padron,mauricio mier padron,100.000000,True
1,jorge julian pereyra escudero,jorge julian pereyra escudero,100.000000,True
2,virgilio cesar valdez portales,yo,50.000000,False
3,jorge rábago ordóñez,NaN,NaN,False
4,rafael covarrubias flores,rafael ovarrubias flores,95.833333,True


In [382]:
df_notario.value_counts()

notario                          notario_pred                   notario_match  success
arturo javier garduño perez      arturo javier garduno perez    96.296296      True       1
cecilio gonzalez marquez         cecilio gonzalez marquez       100.000000     True       1
eduardo adolfo manautou ayala    notario publico                40.000000      False      1
jorge julian pereyra escudero    jorge julian pereyra escudero  100.000000     True       1
maria atala martínez cárdenas    linidos                        28.571429      False      1
mauricio mier padron             mauricio mier padron           100.000000     True       1
rafael covarrubias flores        rafael ovarrubias flores       95.833333      True       1
rita raquel martinez salgado     notario                        42.857143      False      1
salvador sanchez de la barquera  notario                        28.571429      False      1
virgilio cesar valdez portales   yo                             50.000000      False 

In [383]:
occur = df_notario.groupby(['success']).size()
occur

success
False    9
True     5
dtype: int64

In [384]:
df_merge.head()

,documento,company_name_csf,date_pred,notario_pred,society_type_csf,state_csf,company_name,date,notario,society_type,state,society_type_match,company_name_match,notario_match,state_match
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi,100.0,90.909091,100.000000,100.0
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo,100.0,100.000000,100.000000,100.0
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california",100.0,100.000000,50.000000,100.0
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal",100.0,100.000000,NaN,100.0
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco,exelencia ejecutiva premium,13/07/2011,rafael covarrubias flores,sa de cv,"guadalajara, jalisco",100.0,100.000000,95.833333,100.0


In [385]:
df_match = df_merge[match_cols]
df_match.head()

,society_type_match,company_name_match,notario_match,state_match
0,100.0,90.909091,100.000000,100.0
1,100.0,100.000000,100.000000,100.0
2,100.0,100.000000,50.000000,100.0
3,100.0,100.000000,NaN,100.0
4,100.0,100.000000,95.833333,100.0


In [386]:
df_match['success_society_type'] = df_match['society_type_match'].ge(90)
df_match['success_company_name'] = df_match['company_name_match'].ge(90)
df_match['success_state'] = df_match['state_match'].ge(90)
df_match['success_notario'] = df_match['notario_match'].ge(90)

df_match.head()

<ipython-input-386-9e1ed82a1401>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-386-9e1ed82a1401>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-386-9e1ed82a1401>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-386-9e1

,society_type_match,company_name_match,notario_match,state_match,success_society_type,success_company_name,success_state,success_notario
0,100.0,90.909091,100.000000,100.0,True,True,True,True
1,100.0,100.000000,100.000000,100.0,True,True,True,True
2,100.0,100.000000,50.000000,100.0,True,True,True,False
3,100.0,100.000000,NaN,100.0,True,True,True,False
4,100.0,100.000000,95.833333,100.0,True,True,True,True


In [387]:
total = 14

occur_society_type = df_match.groupby(['success_society_type']).size()
occur_company_name = df_match.groupby(['success_company_name']).size()
occur_notario = df_match.groupby(['success_notario']).size()
occur_state = df_match.groupby(['success_state']).size()

print(occur_society_type)
print()

print(occur_company_name)
print()

print(occur_notario)
print()

print(occur_state)

success_society_type
True    14
dtype: int64

success_company_name
True    14
dtype: int64

success_notario
False    9
True     5
dtype: int64

success_state
False     3
True     11
dtype: int64


Society type: 65%

Company name: 100%

Notario: 36%

State: 79%

In [388]:
df_notario

,notario,notario_pred,notario_match,success
0,mauricio mier padron,mauricio mier padron,100.000000,True
1,jorge julian pereyra escudero,jorge julian pereyra escudero,100.000000,True
2,virgilio cesar valdez portales,yo,50.000000,False
3,jorge rábago ordóñez,NaN,NaN,False
4,rafael covarrubias flores,rafael ovarrubias flores,95.833333,True
5,maria atala martínez cárdenas,linidos,28.571429,False
6,enrique almanza pedraza,NaN,NaN,False
7,rene gurmilan sanchez,NaN,NaN,False
8,benjamín arreguín meneses,NaN,NaN,False
9,salvador sanchez de la barquera,notario,28.571429,False
